In [106]:
import numpy as np
import pyspark
import pyspark.sql.functions as fn
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pandas as pd

In [107]:
from IPython.display import display, HTML
display(HTML("<style>pre {white-space: pre !important; }</style>"))

In [108]:
spark = SparkSession.builder.getOrCreate()
spark

In [109]:
df = spark.read.format('csv')\
                .option('header', 'True')\
                .option('inferschema', 'True')\
                .load('Nashville Accidents Jan 2018 - Apl 2025.csv')

In [110]:
df.show(10,truncate=False)

+---------------+---------------------+------------------------+------------------+--------------------+---------------+-----------+---------------------------------------+---------------------+------------------------+------------------------------+---------+-----+--------+----------+----------+------------+-------------------------------+--------+--------+-----+-------+------------------+--------------+-----------------+-------------+------------+
|Accident Number|Date and Time        |Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description             |Weather Description  |Illumination Description|Street Address                |City     |State|Precinct|Lat       |Long      |HarmfulCodes|HarmfulDescriptions            |ObjectId|Zip Code|RPA  |Weather|IlluACCIDEmination|Collision Type|Reporting Officer|x            |y           |
+---------------+---------------------+------------------------+------------------+---------

In [111]:
df.printSchema()

root
 |-- Accident Number: long (nullable = true)
 |-- Date and Time: string (nullable = true)
 |-- Number of Motor Vehicles: integer (nullable = true)
 |-- Number of Injuries: integer (nullable = true)
 |-- Number of Fatalities: integer (nullable = true)
 |-- Property Damage: string (nullable = true)
 |-- Hit and Run: string (nullable = true)
 |-- Collision Type Description: string (nullable = true)
 |-- Weather Description: string (nullable = true)
 |-- Illumination Description: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- HarmfulCodes: string (nullable = true)
 |-- HarmfulDescriptions: string (nullable = true)
 |-- ObjectId: integer (nullable = true)
 |-- Zip Code: integer (nullable = true)
 |-- RPA: integer (nullable = true)
 |-- Weather: integer (nullable = true)
 |-- I

## Finding Null Values in Each Column:

In [112]:
df.count()

216117

In [113]:
def nullcol(df):
    df_null_cols = df.select([sum(col(c).isNull().cast('int')).alias(c) for c in df.columns])
    df_null_cols.show()
nullcol(df)    

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-------+------------------+--------------+-----------------+---+---+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Weather|IlluACCIDEmination|Collision Type|Reporting Officer|  x|  y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------

In [114]:
df.describe().show(truncate=False)

+-------+---------------------+--------------------+------------------------+------------------+---------------------+---------------+-----------+--------------------------+----------------------+------------------------+-----------------------------------+------------+------+--------+-------------------+------------------+------------------+---------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+
|summary|Accident Number      |Date and Time       |Number of Motor Vehicles|Number of Injuries|Number of Fatalities |Property Damage|Hit and Run|Collision Type Description|Weather Description   |Illumination Description|Street Address                     |City        |State |Precinct|Lat                |Long              |HarmfulCodes      |HarmfulDescriptions  |ObjectId          |Zip Code         |RPA               |Weather           |IlluACCIDEmination|Colli

## Get the NUMBER of distinct values for each column:

In [115]:
df_distinct_cols = df.select([countDistinct(c).alias(c) for c in df.columns])
df_distinct_cols.show()

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+-----+-----+------------+-------------------+--------+--------+----+-------+------------------+--------------+-----------------+-----+-----+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|  Lat| Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code| RPA|Weather|IlluACCIDEmination|Collision Type|Reporting Officer|    x|    y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+-----+-----+------------+

## Get the distinct values for each column:

In [116]:
from pyspark.sql import Row
unique_values_dict = {}
distinct_number = 11

for col_name in df.columns:
    values = df.select(col_name).distinct().limit(distinct_number).rdd.flatMap(lambda x: x).collect()
    padded_values = values + [None] * (distinct_number - len(values))
    unique_values_dict[col_name] = padded_values
rows = [Row(**{col: unique_values_dict[col][i] for col in df.columns}) for i in range(distinct_number)]

spark_df_distinct = spark.createDataFrame(rows)

spark_df_distinct.show(truncate=False)

+---------------+---------------------+------------------------+------------------+--------------------+---------------+-----------+---------------------------------------+---------------------+------------------------+-------------------------------+--------------+-----+--------+--------+---------+------------+-------------------------------------------------+--------+--------+-----+-------+------------------+--------------+-----------------+-------------+------------+
|Accident Number|Date and Time        |Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description             |Weather Description  |Illumination Description|Street Address                 |City          |State|Precinct|Lat     |Long     |HarmfulCodes|HarmfulDescriptions                              |ObjectId|Zip Code|RPA  |Weather|IlluACCIDEmination|Collision Type|Reporting Officer|x            |y           |
+---------------+---------------------+-----------

### Get the number of null rows:

In [117]:
from functools import reduce
null_condition = reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns))
null_row_count = df.filter(null_condition).count()
print(null_row_count)
print(df.count() - null_row_count)

200442
15675


## Filling null with Median:
#### (Number of Motor Vehicles, Number of injuries, Number of Fatalities)

In [118]:
median_motor_vecheils = df.select(fn.median(col('Number of Motor Vehicles'))).collect()
median_Number_of_injuries = df.select(fn.median(col('Number of Injuries'))).collect()
median_Number_of_Fatalities = df.select(fn.median(col('Number of Fatalities'))).collect()

median_dict = {
    "Number of Motor Vehicles": median_motor_vecheils[0][0],
    "Number of Injuries": median_Number_of_injuries[0][0],
    "Number of Fatalities": median_Number_of_Fatalities[0][0]
}

df_fill_na = df.na.fill(median_dict)
nullcol(df_fill_na)

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-------+------------------+--------------+-----------------+---+---+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Weather|IlluACCIDEmination|Collision Type|Reporting Officer|  x|  y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------

## Filling Null With "UNKNOWN":

In [119]:
df_fill_na.count()

216117

In [120]:
df_fill_na_cat = df_fill_na.fillna({"Collision Type Description": "UNKNOWN",
                         "Weather Description":"UNKNOWN",
                         "Illumination Description":"UNKNOWN",
                          "Street Address": "UNKNOWN",
                        "HarmfulCodes":"UNKNOWN",
                         "HarmfulDescriptions":"UNKNOWN",
                        })
nullcol(df_fill_na_cat)

#drop Weather|IlluACCIDEmination

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-------+------------------+--------------+-----------------+---+---+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Weather|IlluACCIDEmination|Collision Type|Reporting Officer|  x|  y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------

## Drop Nulls:

In [122]:
columns_to_drop = ["Weather", "IlluACCIDEmination","Collision Type"]
df_drop_null_columns = df_fill_na_cat.drop(*columns_to_drop)

In [123]:
nullcol(df_drop_null_columns)

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+
|              0|            0|   

In [124]:
df_drop_null_columns.count()

216117

In [125]:
df_drop_null_columns.groupBy("Collision Type Description", "Property Damage").count().orderBy("Collision Type Description").show(30,truncate=False)

+---------------------------------------+---------------+-----+
|Collision Type Description             |Property Damage|count|
+---------------------------------------+---------------+-----+
|ANGLE                                  |NULL           |52536|
|ANGLE                                  |Y              |1679 |
|Front to Rear                          |Y              |696  |
|Front to Rear                          |NULL           |65571|
|HEAD-ON                                |Y              |224  |
|HEAD-ON                                |NULL           |5031 |
|NOT COLLISION W/MOTOR VEHICLE-TRANSPORT|Y              |13531|
|NOT COLLISION W/MOTOR VEHICLE-TRANSPORT|NULL           |23004|
|NOT COLLISION W/MOTOR VEHICLE-TRANSPORT|N              |1    |
|OTHER                                  |Y              |95   |
|OTHER                                  |NULL           |2221 |
|REAR END                               |NULL           |1    |
|REAR-TO-REAR                           

In [130]:
nullcol(df_drop_null_columns)

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+
|              0|            0|   

## Handeling Nulls in Property Damage inrespective to "Collision Type Description":

In [131]:
df_drop_null_columns.createOrReplaceTempView("PropertyDamage")
df_handle_property_damage = spark.sql("""
SELECT 
    *,
    CASE 
        WHEN `Property Damage` IS NOT NULL THEN `Property Damage`
        WHEN `Collision Type Description` IN (
            'REAR-TO-REAR', 'Front to Rear', 'HEAD-ON', 
            'ANGLE', 'Rear to Side', 'NOT COLLISION W/MOTOR VEHICLE-TRANSPORT'
        ) THEN 'Yes'
        WHEN `Collision Type Description` = 'SIDESWIPE - OPPOSITE DIRECTION' THEN 'Possible'
        ELSE 'No'
    END AS `Property Damage Imputed`
FROM PropertyDamage
""")

In [132]:
nullcol(df_handle_property_damage)

+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+-----------------------+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|Property Damage Imputed|
+---------------+-------------+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-------------

In [134]:
df_drop_property_damage = df_handle_property_damage.drop("Property Damage")

In [135]:
nullcol(df_drop_property_damage)

+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+-----------------------+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|Property Damage Imputed|
+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+-----------------------+
|         

In [136]:
df_rename_property_damage = df_drop_property_damage.withColumnRenamed("Property Damage Imputed", "Property Damage")

In [137]:
nullcol(df_rename_property_damage)

+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|Property Damage|
+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|              0|            0|   

In [ ]:
df_rename_property_damage.select(df['Property Damage']).where(col('Property Damage') == 'Y').count()

In [ ]:
df_rename_property_damage.select(df['Property Damage']).where(col('Property Damage') == 'N').count()

In [138]:
df_yes_no_property_damage = df_rename_property_damage.withColumn(
    "Property Damage",
    when(col("Property Damage") == "Y", "Yes")
    .when(col("Property Damage") == "N", "No")
    .otherwise(col("Property Damage")))

In [139]:
df_yes_no_property_damage.count()

216117

In [140]:
nullcol(df_yes_no_property_damage)

+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|Property Damage|
+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|              0|            0|   

In [141]:
df_cleaned = df_yes_no_property_damage.dropna()

In [142]:
df_cleaned.count()

215939

In [143]:
nullcol(df_cleaned)

+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|Accident Number|Date and Time|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|Property Damage|
+---------------+-------------+------------------------+------------------+--------------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+---------------+
|              0|            0|   

In [144]:
df_cleaned.show(truncate=False)

+---------------+---------------------+------------------------+------------------+--------------------+-----------+---------------------------------------+---------------------+------------------------+-----------------------------------+---------+-----+--------+----------+----------+------------+-----------------------------------------------+--------+--------+-----+-----------------+-------------+------------+---------------+
|Accident Number|Date and Time        |Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Hit and Run|Collision Type Description             |Weather Description  |Illumination Description|Street Address                     |City     |State|Precinct|Lat       |Long      |HarmfulCodes|HarmfulDescriptions                            |ObjectId|Zip Code|RPA  |Reporting Officer|x            |y           |Property Damage|
+---------------+---------------------+------------------------+------------------+--------------------+-----------+------------------

In [145]:
df_cleaned.printSchema()

root
 |-- Accident Number: long (nullable = true)
 |-- Date and Time: string (nullable = true)
 |-- Number of Motor Vehicles: integer (nullable = true)
 |-- Number of Injuries: integer (nullable = true)
 |-- Number of Fatalities: integer (nullable = true)
 |-- Hit and Run: string (nullable = true)
 |-- Collision Type Description: string (nullable = false)
 |-- Weather Description: string (nullable = false)
 |-- Illumination Description: string (nullable = false)
 |-- Street Address: string (nullable = false)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- HarmfulCodes: string (nullable = false)
 |-- HarmfulDescriptions: string (nullable = false)
 |-- ObjectId: integer (nullable = true)
 |-- Zip Code: integer (nullable = true)
 |-- RPA: integer (nullable = true)
 |-- Reporting Officer: integer (nullable = true)
 |-- x: double (nullable = true)
 |--

## Extract DateTime data:

In [146]:
df2 = df_cleaned.withColumn("PM/AM", fn.substring(col("Date and Time"),-2,2))

In [147]:
df2 = df2.withColumn("Date and Time", to_timestamp("Date and Time", "M/d/yyyy h:mm:ss a"))

In [148]:
df_final = df2.withColumn("year", year("Date and Time")) \
       .withColumn("month", month("Date and Time"))\
       .withColumn("day", dayofmonth("Date and Time"))\
       .withColumn("Time", date_format("Date and Time", "HH:mm:ss"))\
       .withColumn("hour", hour("Date and Time")) \
       .withColumn("minutes", minute("Date and Time"))

In [149]:
df_final.columns

['Accident Number',
 'Date and Time',
 'Number of Motor Vehicles',
 'Number of Injuries',
 'Number of Fatalities',
 'Hit and Run',
 'Collision Type Description',
 'Weather Description',
 'Illumination Description',
 'Street Address',
 'City',
 'State',
 'Precinct',
 'Lat',
 'Long',
 'HarmfulCodes',
 'HarmfulDescriptions',
 'ObjectId',
 'Zip Code',
 'RPA',
 'Reporting Officer',
 'x',
 'y',
 'Property Damage',
 'PM/AM',
 'year',
 'month',
 'day',
 'Time',
 'hour',
 'minutes']

In [150]:
df_final = df_final.select(['Accident Number',
 'Date and Time',
 'year',
 'month',
 'day',
 'Time',
 'hour',
 'minutes',
 'PM/AM',
 'Number of Motor Vehicles',
 'Number of Injuries',
 'Number of Fatalities',
 'Property Damage',
 'Hit and Run',
 'Collision Type Description',
 'Weather Description',
 'Illumination Description',
 'Street Address',
 'City',
 'State',
 'Precinct',
 'Lat',
 'Long',
 'HarmfulCodes',
 'HarmfulDescriptions',
 'ObjectId',
 'Zip Code',
 'RPA',
 'Reporting Officer',
 'x',
 'y'
 ])

In [151]:
df_final.show()

+---------------+-------------------+----+-----+---+--------+----+-------+-----+------------------------+------------------+--------------------+---------------+-----------+--------------------------+--------------------+------------------------+--------------------+---------+-----+--------+----------+----------+------------+--------------------+--------+--------+-----+-----------------+-------------+------------+
|Accident Number|      Date and Time|year|month|day|    Time|hour|minutes|PM/AM|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description| Weather Description|Illumination Description|      Street Address|     City|State|Precinct|       Lat|      Long|HarmfulCodes| HarmfulDescriptions|ObjectId|Zip Code|  RPA|Reporting Officer|            x|           y|
+---------------+-------------------+----+-----+---+--------+----+-------+-----+------------------------+------------------+--------------------+---------------+---

In [152]:
df_final.printSchema()

root
 |-- Accident Number: long (nullable = true)
 |-- Date and Time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- Time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- PM/AM: string (nullable = true)
 |-- Number of Motor Vehicles: integer (nullable = true)
 |-- Number of Injuries: integer (nullable = true)
 |-- Number of Fatalities: integer (nullable = true)
 |-- Property Damage: string (nullable = true)
 |-- Hit and Run: string (nullable = true)
 |-- Collision Type Description: string (nullable = false)
 |-- Weather Description: string (nullable = false)
 |-- Illumination Description: string (nullable = false)
 |-- Street Address: string (nullable = false)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Precinct: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable

In [153]:
nullcol(df_final)

+---------------+-------------+----+-----+---+----+----+-------+-----+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----+------------+-------------------+--------+--------+---+-----------------+---+---+
|Accident Number|Date and Time|year|month|day|Time|hour|minutes|PM/AM|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description|Weather Description|Illumination Description|Street Address|City|State|Precinct|Lat|Long|HarmfulCodes|HarmfulDescriptions|ObjectId|Zip Code|RPA|Reporting Officer|  x|  y|
+---------------+-------------+----+-----+---+----+----+-------+-----+------------------------+------------------+--------------------+---------------+-----------+--------------------------+-------------------+------------------------+--------------+----+-----+--------+---+----

## Handle the comma ',' in Columns:

In [154]:
string_columns = [field.name for field in df_final.schema.fields if isinstance(field.dataType, StringType)]

for column in string_columns:
    df_final = df_final.withColumn(column, regexp_replace(col(column), ",", ";"))

In [51]:
df_final.count()

215939

In [52]:
len(df_final.columns)

31

In [55]:
df_final.show()

+---------------+-------------------+----+-----+---+--------+----+-------+-----+------------------------+------------------+--------------------+---------------+-----------+--------------------------+--------------------+------------------------+--------------------+---------+-----+--------+----------+----------+------------+--------------------+--------+--------+-----+-----------------+-------------+------------+
|Accident Number|      Date and Time|year|month|day|    Time|hour|minutes|PM/AM|Number of Motor Vehicles|Number of Injuries|Number of Fatalities|Property Damage|Hit and Run|Collision Type Description| Weather Description|Illumination Description|      Street Address|     City|State|Precinct|       Lat|      Long|HarmfulCodes| HarmfulDescriptions|ObjectId|Zip Code|  RPA|Reporting Officer|            x|           y|
+---------------+-------------------+----+-----+---+--------+----+-------+-----+------------------------+------------------+--------------------+---------------+---

In [ ]:
df_final.coalesce(1).write.mode("overwrite").option("header", True).csv("Nashville Accidents.csv")